In [3]:
import tarfile
import json
from bz2 import BZ2File


def extract_jsonlist(tar_path, file_name):
    with tarfile.open(tar_path, mode='r') as tar:
        bz2_file = tar.extractfile(file_name)
        return [json.loads(line.decode('utf-8')) for line in BZ2File(bz2_file)]


# File paths
fname = "../cmv.tar.bz2"
train_fname = "op_task/train_op_data.jsonlist.bz2"
test_fname = "op_task/heldout_op_data.jsonlist.bz2"

# Extract and deserialize the JSON lists
original_posts_train = extract_jsonlist(fname, train_fname)
original_posts_test = extract_jsonlist(fname, test_fname)


def cleanup(cmv_post):
    lines = [line for line in cmv_post.splitlines()
             if not line.lstrip().startswith("&gt;")
             and not line.lstrip().startswith("____")
             and "edit" not in " ".join(line.lower().split()[:2])
             ]
    return "\n".join(lines)


def clean_dataset(dataset):
    for i in range(len(dataset)):
        dataset[i]['selftext'] = cleanup(dataset[i]['selftext'])
    return dataset


original_posts_train = clean_dataset(original_posts_train)
original_posts_test = clean_dataset(original_posts_test)

In [26]:
def create_balanced_dataset(dataset):
    import pandas as pd
    df = pd.DataFrame(dataset, columns=['title', 'delta_label', 'name', 'selftext'])
    is_malleable = df['delta_label']
    n_false, n_true = is_malleable.value_counts()
    min_n_samples = min(n_false, n_true)
    malleable = df[is_malleable].sample(n=min_n_samples,
                                        random_state=42)
    not_malleable = df[~is_malleable].sample(n=min_n_samples,
                                             random_state=42)
    df_balanced = pd.concat([malleable, not_malleable])
    # shuffle the dataframe
    df_balanced = df_balanced.sample(frac=1).reset_index(drop=True)
    return df_balanced

original_posts_train = create_balanced_dataset(original_posts_train)
original_posts_test = create_balanced_dataset(original_posts_test)

delta_label
False    3191
True     3191
Name: count, dtype: int64

In [10]:
x = [1,2,3,4,5]
i = [True, True, True, False, True]

TypeError: bad operand type for unary ~: 'list'